# Retrieve player info for analysis

## Import relevant packages

In [10]:
import pandas as pd
from nba_api.stats.endpoints import commonallplayers, playergamelog
import time

import warnings
warnings.filterwarnings("ignore")

## Fetch the number of minutes that each player played and the team they played on

In [11]:
# Step 1: Fetch all players who were active during the 2015-16 season
players_2015_16 = commonallplayers.CommonAllPlayers(is_only_current_season=0, league_id='00', season='2015-16').get_data_frames()[0]
active_players = players_2015_16[players_2015_16['ROSTERSTATUS'] == 1]

# Step 2: Loop through active players and fetch their game logs for the 2015-16 season
player_minutes = []

for _, player in active_players.iterrows():
    try:
        # Fetch game logs for the player in the 2015-16 season
        game_log = playergamelog.PlayerGameLog(player_id=player['PERSON_ID'], season='2015-16').get_data_frames()[0]
        
        # Filter for games contained in the dataset
        game_log['GAME_DATE'] = pd.to_datetime(game_log['GAME_DATE'])
        pre_all_star_games = game_log[game_log['GAME_DATE'] <= '2016-01-23']
        
        if not pre_all_star_games.empty:
            # Calculate total minutes before the All-Star break
            total_minutes = pre_all_star_games['MIN'].astype(int).sum()
            
            # Extract the team abbreviation from the MATCHUP column
            pre_all_star_games['TEAM_ABBREVIATION'] = pre_all_star_games['MATCHUP'].str.split().str[0]
            team_abbreviation = pre_all_star_games['TEAM_ABBREVIATION'].iloc[0]
            
            # Store the result as a tuple of player name, team, and total minutes
            player_minutes.append((player['PERSON_ID'], player['DISPLAY_FIRST_LAST'], team_abbreviation, total_minutes))
    
    except Exception as e:
        print(f"Error fetching data for {player['DISPLAY_FIRST_LAST']}: {e}")
    time.sleep(1)

# Step 3: Convert the result into a DataFrame for easier analysis
minutes_df = pd.DataFrame(player_minutes, columns=['Player ID', 'Player Name', 'Team', 'Total Minutes'])

# Step 4: Sort by total minutes and view the top players per team
minutes_df.sort_values(by='Total Minutes', ascending=False)

,Player ID,Player Name,Team,Total Minutes
154,201935,James Harden,HOU,1682
263,203114,Khris Middleton,MIL,1656
65,203484,Kentavious Caldwell-Pope,DET,1648
62,202710,Jimmy Butler,CHI,1642
393,202689,Kemba Walker,CHA,1580
...,...,...,...,...
400,203963,Shayne Whittington,IND,7
96,1626155,Sam Dekker,HOU,6
262,1626175,Jordan Mickey,BOS,6
120,1626245,Cristiano Felicio,CHI,5


## Get the top 8 players in terms of minutes for each team

In [12]:
n=8
top_players_per_team = minutes_df.groupby('Team').apply(lambda x: x.nlargest(n, 'Total Minutes')).reset_index(drop=True)
top_players_per_team

,Player ID,Player Name,Team,Total Minutes
0,200794,Paul Millsap,ATL,1443
1,201143,Al Horford,ATL,1420
2,2594,Kyle Korver,ATL,1310
3,201952,Jeff Teague,ATL,1196
4,203145,Kent Bazemore,ATL,1131
...,...,...,...,...
235,203490,Otto Porter Jr.,WAS,1089
236,202066,Garrett Temple,WAS,952
237,201196,Ramon Sessions,WAS,872
238,203078,Bradley Beal,WAS,716


## Get the position for every player